In [1]:
#RDDMS demo via Rest-APIs 
#documented in the SWAGGER pages
#http://10.0.1.4:8003/Reservoir/v2#/Resources/

In [2]:
#Libs needed
!pip install requests
!pip install pandas
!pip install --upgrade matplotlib

In [3]:
import requests
import json
import urllib.parse
import pandas as pd
import numpy as np

In [4]:
#Health check

In [5]:
headers = {
    'accept': 'application/json',
}

response = requests.get('http://10.0.1.4:8003/Reservoir/v2/health/readiness', headers=headers)

assert response.text == 'true', 'RDDMS service NOT up and running'

In [6]:
#Get Baeren Token 

In [7]:
headers = {
    'accept': 'application/json',
}

response = requests.get('http://10.0.1.4:8003/Reservoir/v2/auth/token', headers=headers)

In [8]:
bearen_key = response.json()['token']
print('Bearer Token:',bearen_key)

In [9]:
#List the available data space(s)

In [10]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearen_key}',
}

params = {
    '$skip': '0',
    '$top': '30',
}

response = requests.get('http://10.0.1.4:8003/Reservoir/v2/dataspaces', params=params, headers=headers)

In [11]:
print(*response, sep='\n')
print('-----------------')

SelectedDataspace = 4

#You need to URL encode the namespace
print('Dataspace name:', response.json()[SelectedDataspace]['path'])
dataspace_name = urllib.parse.quote(response.json()[SelectedDataspace]['path'], safe="")
print('URL encoded namespace:',dataspace_name)

In [12]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearen_key}'
}

response = requests.get(f'http://10.0.1.4:8003/Reservoir/v2/dataspaces/{dataspace_name}/resources?%24skip=0&%24top=20', headers=headers)

In [13]:
#print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

In [14]:
# List all object given an object type

In [15]:
response = requests.get(
    f'http://10.0.1.4:8003/Reservoir/v2/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation',
    params=params,
    headers=headers,
)

In [16]:
print(json.dumps(response.json(),indent=4))

In [17]:
SelectedSurface =  0 # fisrt and only object in dataspace
uuid = urllib.parse.quote(response.json()[SelectedSurface]['uri'].split('(')[-1].replace(')','')) 

print('uuid:', uuid)

In [18]:
print(json.dumps(response.json(),indent=4))
response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
print(response_df)

In [19]:
#response_df['uri']

In [20]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearen_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}

response = requests.get(
    f'http://10.0.1.4:8003/Reservoir/v2/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

In [21]:
headers = {
    'accept': 'application/x-resqml+xml',
    'Authorization': f'Bearer {bearen_key}'
}

params = {
    '$format': 'json',
    'arrayMetadata': 'false',
    'arrayValues': 'false',
    'referencedContent': 'true',
}


response = requests.get(
    f'http://10.0.1.4:8003/Reservoir/v2/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays',
    params=params,
    headers=headers,
)
print(json.dumps(response.json(),indent=4))

In [22]:
uuid_url = urllib.parse.quote(response.json()[0]['uid']['pathInResource'], safe="")
print('uuid_url:', uuid_url)

In [23]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {bearen_key}'
    }

params = {
    'format': 'json'
}

response = requests.get(
    f'http://10.0.1.4:8003/Reservoir/v2/dataspaces/{dataspace_name}/resources/resqml20.obj_Grid2dRepresentation/{uuid}/arrays/{uuid_url}',
    params=params,
    headers=headers,
)

In [24]:
print(response.text)

In [25]:
print('Data length:',len(response.json()['data']['data']))
print('Dimensions:',response.json()['data']['dimensions'])
assert len(response.json()['data']['data']) == ( int(response.json()['data']['dimensions'][0]) * int(response.json()['data']['dimensions'][1]) )

In [26]:
xy_array = np.reshape(response.json()['data']['data'], (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1]))).T

In [27]:
len(xy_array[:,:])

In [28]:
z = np.array(response.json()['data']['data'], dtype=np.float32)
z_array = np.reshape(z, (int(response.json()['data']['dimensions'][0]), int(response.json()['data']['dimensions'][1])))
print(xy_array.shape)

In [29]:
#https://matplotlib.org/stable/gallery/mplot3d/custom_shaded_3d_surface.html#sphx-glr-gallery-mplot3d-custom-shaded-3d-surface-py
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import numpy as np

# Load and format data
#dem = cbook.get_sample_data('jacksboro_fault_dem.npz', np_load=True)
#z = dem['elevation']
z = z_array
nrows, ncols = z.shape
print(nrows, ncols)
#x = np.linspace(dem['xmin'], dem['xmax'], ncols)
#y = np.linspace(dem['ymin'], dem['ymax'], nrows)
x = np.linspace(0, 5, ncols)
y = np.linspace(0, 5, nrows)
x, y = np.meshgrid(x, y)
#region = np.s_[5:50, 5:50]
#region = np.s_[:, :]
#x, y, z = x[region], y[region], z[region]

# Set up plot
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))


ls = LightSource(360, 45)
# To use a custom hillshading mode, override the built-in shading and pass
# in the rgb colors of the shaded surface calculated from "shade".
rgb = ls.shade(z, cmap=cm.gist_earth, vert_exag=0.1)
surf = ax.plot_surface(y, x, z, cmap = cm.gist_earth,linewidth=0, antialiased=False, shade=False)
#
#surf = ax.plot_surface(x, y, z)

plt.show()